In [ ]:
import pandas as pd

municipios = ["tlaquepaque", "tonala", "tlajomulco", "guadalajara", "zapopan"]
propiedades_jalisco = pd.read_csv("Data/jalisco/propiedades_jalisco.csv")

dfs = []
# Unimos los datos de las variables de entorno de cada municipio
for municipio in municipios:
    df_municipio = pd.read_csv(f"Data/jalisco/direcciones_lineales/direcciones_lineales_{municipio}.csv")
    dfs.append(df_municipio)
df_entorno = pd.concat(dfs)
df_entorno

In [ ]:
df_estructurales = pd.read_csv("Data/jalisco/propiedades_jalisco.csv")

df = df_estructurales.merge(df_entorno, on="direccion")

# Eliminamos las filas con valores nulos
df = df.dropna()
df.describe()

In [ ]:
import matplotlib.pyplot as plt

df["municipio"].value_counts().plot.pie(autopct='%1.1f%%', figsize=(6, 6), title='Distribución')
plt.plot()

In [ ]:
df["jardin"].value_counts().plot.pie(autopct='%1.1f%%', figsize=(6, 6), title='Distribución')
plt.plot()

In [ ]:
import plotly.express as px

px.box(df, y=['precio'], title='Distribución de precio')


In [ ]:
from scipy.stats import zscore

# Eliminamos la propiedad de $125 MXN
df = df[df['precio'] != 125]
zscores_precio = zscore(df['precio'])

df = df[abs(zscores_precio) < 3]
df["precio"].describe()

In [ ]:
df[df["precio"] <= 400000]

In [ ]:
import seaborn as sns

sns.violinplot(df["precio"], inner="box")
plt.show()

Repetimos el ejercicio para la variable `m2_lote` que también presenta valores atípicos.

In [ ]:
px.box(df, y=['m2_lote'], title='Distribución de m2_lote')

In [ ]:
df = df[df['m2_lote'] != 1110000]
zscores_lote = zscore(df['m2_lote'])

df = df[abs(zscores_lote) < 3]
df["m2_lote"].describe()

In [ ]:
sns.violinplot(df["m2_lote"], inner="box")

Finalmente, repetimos el procedimiento para la variable `Distancia la centro de la ciudad (km)` que presenta un valor atípico.

In [ ]:
px.box(df, y=['Distancia al centro de la ciudad (km)'], title='Distribución de distancia al centro de la ciudad (km)')

In [ ]:
df = df[df['Distancia al centro de la ciudad (km)'] != 7782.9]
zscores_centro = zscore(df['Distancia al centro de la ciudad (km)'])

df = df[abs(zscores_centro) < 3]
df["Distancia al centro de la ciudad (km)"].describe()

In [ ]:
sns.violinplot(df["Distancia al centro de la ciudad (km)"], inner="box")

In [ ]:
df

In [ ]:
df["municipio"].value_counts().plot.pie(autopct='%1.1f%%', figsize=(6, 6), title='Distribución')
plt.plot()

In [ ]:
# Agregamos los indices de criminalidad

indices_criminalidad = {
    'guadalajara': {'robo_de_vehiculo_automotor':192.69, 'robo_a_negocio':80.67, 'robo_casa_habitacion':21.82},
    'tlaquepaque': {'robo_de_vehiculo_automotor':220.06, 'robo_a_negocio':67.25, 'robo_casa_habitacion':45.35},
    'tonala': {'robo_de_vehiculo_automotor':108.46, 'robo_a_negocio':33.36, 'robo_casa_habitacion':16.76},
    'tlajomulco': {'robo_de_vehiculo_automotor':91.67, 'robo_a_negocio':67.88, 'robo_casa_habitacion':58.60},
    'zapopan': {'robo_de_vehiculo_automotor':97.58, 'robo_a_negocio':44.57, 'robo_casa_habitacion':16.27}
}

# Convertir a DataFrame y poner municipio como columna
df_criminalidad = pd.DataFrame.from_dict(indices_criminalidad, orient='index').reset_index()
df_criminalidad = df_criminalidad.rename(columns={'index':'municipio'})

df = df.merge(df_criminalidad, on='municipio', how='left')
df

In [ ]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

# Separar variables
X = df[['m2_lote',  #x1
        'banos',    #x2
        'estacionamientos', #x3
        'recamaras',    #x4
        'jardin',   #x5
        'Distancia al parque (km)', #x6
        'Distancia a plaza (km)',   #x7
        'Distancia al centro de la ciudad (km)',    #x8
        'robo_de_vehiculo_automotor',   #x9
        'robo_a_negocio',   #x10
        'robo_casa_habitacion']]    #x11
y = df['precio']

# Estandarizar X y y
scaler_X = StandardScaler()
# scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
# y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Agregar constante (intercepto)
X_scaled = sm.add_constant(X_scaled)

# Ajustar modelo MCO
modelo = sm.OLS(y, X_scaled).fit()

# Resultados
print(modelo.summary())


In [ ]:
# Obtén el coeficiente del modelo
coef_robo_vehiculo = modelo.params[modelo.model.exog_names.index('x11')]  # o el nombre correcto si lo renombras

# Obtén la desviación estándar original del predictor
std_robo_vehiculo = df['robo_casa_habitacion'].std()

# Calcula el impacto por unidad
impacto_por_robo = coef_robo_vehiculo / std_robo_vehiculo

print(f"Cambio estimado en precio por cada 1 robo adicional por 100k hab: ${impacto_por_robo:,.0f} pesos")

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]
for x in vif:
    print(x)